In [1]:
import os 
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset
from torchvision import transforms,models

In [3]:
class BrainTumorDataset(Dataset):
    def __init__(self, data_dir, classes, transform=None):
        self.data_dir = data_dir
        self.classes = classes
        self.transform = transform

        # collect image paths and labels
        self.image_paths = []
        self.labels = []
        for label, class_name in enumerate(classes):
            class_dir = os.path.join(data_dir, class_name)
            for file in os.listdir(class_dir):
                if file.lower().endswith((".jpg", ".png", ".jpeg")):
                    self.image_paths.append(os.path.join(class_dir, file))
                    self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]

        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label


In [4]:
classes=["glioma","meningioma","pituitary","notumor"]

data_dir="./brain_dataset"
train_dir=os.path.join(data_dir,"Training")
test_dir=os.path.join(data_dir,"Testing")



In [5]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std=[0.229,0.224,0.225]),
])
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std=[0.229,0.224,0.225]),
])

train_ds = BrainTumorDataset(train_dir, classes, transform=train_transform)
test_ds = BrainTumorDataset(test_dir, classes, transform=val_transform)

num_workers = os.cpu_count() 

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=num_workers)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False, num_workers=num_workers)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [6]:
model=models.resnet18(pretrained=True)
num_ftrs=model.fc.in_features
model.fc=nn.Linear(num_ftrs,len(classes))
model=model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=1e-4)
scheduler=optim.lr_scheduler.ReduceLROnPlateau(optimizer,"min",patience=3)


c:\Users\vedan\.conda\envs\dl-gpu\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\vedan\.conda\envs\dl-gpu\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
best_val_loss=float("inf")
num_epochs=10

In [ ]:
from tqdm import tqdm  # install with `pip install tqdm`

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_corrects = 0

    # Wrap train_loader with tqdm for progress bar
    train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch")

    for inputs, labels in train_pbar:
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        running_corrects += torch.sum(preds == labels)

        # Update tqdm bar with current loss/acc
        train_pbar.set_postfix({
            "loss": loss.item(),
            "acc": (running_corrects.double() / (len(train_pbar) * train_loader.batch_size)).item()
        })

    epoch_loss = running_loss / len(train_ds)
    epoch_acc = running_corrects.double() / len(train_ds)

    # Validation
    model.eval()
    val_loss = 0.0
    val_corrects = 0

    with torch.no_grad():
        val_pbar = tqdm(test_loader, desc="Validating", unit="batch")
        for inputs, labels in val_pbar:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)

            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            val_corrects += torch.sum(preds == labels)

    val_loss = val_loss / len(test_ds)
    val_acc = val_corrects.double() / len(test_ds)

    print(f"\nEpoch {epoch+1}/{num_epochs} "
          f"train_loss={epoch_loss:.4f} train_acc={epoch_acc:.4f} "
          f"val_loss={val_loss:.4f} val_acc={val_acc:.4f}")

    scheduler.step(val_loss)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_brain_tumor_resnet18.pth")
        print("Saved best model.")


Epoch 1/10:   0%|          | 0/179 [00:00<?, ?batch/s]

In [ ]:
model.load_state_dict(torch.load("best_brain_tumor_resnet18.pth"))
model.eval()
test_corrects=0
total=0

with torch.no_grad():
    for inputs,labels in test_loader:
        inputs=inputs.to(device)
        labels=labels.to(device)
        outputs=model(inputs)
        _,preds=torch.max(outputs,1)
        test_corrects+=torch.sum(preds==labels.data)
        total+=labels.size(0)

test_acc=test_corrects.double()/total
print("Test accuracy:", test_acc.item())